In [ ]:
import pandas as pd
import numpy as np

from gensim.models import Word2Vec
import math
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, LSTM, TimeDistributed, RepeatVector
from keras.utils import np_utils
from keras.models import Model
from keras.wrappers.scikit_learn import KerasClassifier
from keras import backend as K
from keras.layers import Bidirectional
from sklearn.model_selection import train_test_split

column_names = ['type','title','text']
df = pd.read_csv('../all_after_mapping.tsv',sep='\t',names=column_names)
labels = df['type'].values
labels = np.array(labels)
np.save('labels',labels)

tokenlizeword = np.load('../tokenlizeword0225_nopunct.npy',allow_pickle=True)
wmodel=Word2Vec.load('../word2vec.model')

max_size = 512
x_train = []
for k in range(tokenlizeword.shape[0]):
# every article have max_size * 300 embedding matrix
    embedding_matrix = np.zeros((max_size,300))
    for i in range(len(tokenlizeword[k])):
        if(i>=max_size):
            break
        embedding_matrix[i] = wmodel[tokenlizeword[k][i]]
    x_train.append(embedding_matrix)

print("embedding done")
x_train = np.array(x_train)

print(x_train.shape)

x = x_train
y = labels
print(y.shape)
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.1,random_state=5)
y_train = np_utils.to_categorical(y_train)

# !ls
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

model = Sequential()
model.add(Bidirectional(LSTM(200, return_sequences=False), input_shape=(max_size, 300)))
# model.add(TimeDistributed(Dense(7, activation='softmax')))
model.add(Dense(50,activation='relu'))
model.add(Dense(7,activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

model.fit(x_train, y_train, epochs=5,verbose=1,validation_split=0.2)

pred = model.predict(x_test)
pred = np.argmax(pred, axis = 1) # Decode softmax output
accuracy = (pred == y_test).mean()
print('Your test accuracy is %.6f' % (accuracy * 100))



df_bert = pd.DataFrame({
    'id':np.arange(df.shape[0]),
    'label':df['type'],
    'alpha':['a']*df.shape[0],
    'text': df['text'],
})

dfdev = df_bert.iloc[:5000]
dftest = df_bert[5000:10000]
dftrain = df_bert[10000:]


testans = dftest['label']
testans = np.array(testans.values)
np.save('testans',testans)

dftrain.to_csv('./train.tsv', sep='\t', index=False, header=False)
dfdev.to_csv('./dev.tsv', sep='\t', index=False, header=False)
dftest.drop(['label','alpha'],axis=1,inplace=True)
dftest.to_csv('./test.tsv', sep='\t', index=False, header=True)

columns = ['0','1','2','3','4','5','6']
preddf = pd.read_csv('../bert_predict/test_results.tsv',sep='\t',names=columns)

arr = preddf.to_numpy()

arr = arr.argmax(axis=1)

print(arr.shape)
print(testans.shape)
accuracy = (arr == testans).mean()
print('Your test accuracy is %.6f' % (accuracy * 100))




Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:33: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


embedding done
(35546, 512, 300)
(35546,)
(31991, 512, 300)
(31991, 7)
(3555, 512, 300)
(3555,)
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_1 (Bidirection (None, 400)               801600    
_________________________________________________________________
dense_1 (Dense)              (None, 50)                20050     
_________________________________________________________________
dense_2 (Dense)              (None, 7)                 357       
Total params: 822,007
Trainable params: 822,007
Non-trainable params: 0
_________________________________________________________________

Train on 25592 samples, validate on 6399 samples
Epoch 1/5
11360/25592 [============>.................] - ETA: 6:04 - loss: 0.6717 - accuracy: 0.7739